TensorFlow 和 PyTorch 版本的 CNN 實作，來進行圖像分類，並應用 MNIST 和 CIFAR-10 資料集。

### **TensorFlow 實作 CNN（MNIST 資料集）**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# 下載並預處理 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 將訓練和測試圖像數據重塑為 (28, 28, 1) 並正規化到 0-1 範圍
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255.0  # 增加通道維度，並正規化
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255.0

# 建立 CNN 模型
model = models.Sequential([
    # 第一個卷積層，使用 32 個 3x3 的卷積核，激活函數為 ReLU，輸入形狀為 (28, 28, 1)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    # 最大池化層，池化窗口大小為 2x2
    layers.MaxPooling2D((2, 2)),
    # 第二個卷積層，使用 64 個 3x3 的卷積核，激活函數為 ReLU
    layers.Conv2D(64, (3, 3), activation='relu'),
    # 最大池化層，池化窗口大小為 2x2
    layers.MaxPooling2D((2, 2)),
    # 展開層，將多維數據展開為一維
    layers.Flatten(),
    # 全連接層，包含 64 個神經元，激活函數為 ReLU
    layers.Dense(64, activation='relu'),
    # 輸出層，包含 10 個神經元，對應 10 個類別，激活函數為 softmax
    layers.Dense(10, activation='softmax')
])

# 編譯模型
# 使用 Adam 優化器，損失函數為 sparse_categorical_crossentropy，評估指標為準確率
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
# 使用訓練數據進行訓練，訓練 5 個 epoch，每批次大小為 64
model.fit(x_train, y_train, epochs=5, batch_size=64)

# 評估模型
# 使用測試數據進行評估，並輸出測試準確率
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')


### **PyTorch 實作 CNN（MNIST 資料集）**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 資料轉換與預處理
# 將圖像轉換為張量並標準化到 [-1, 1] 範圍
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 下載並加載 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 使用 DataLoader 將資料集分批次加載
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 創建 CNN 模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一個卷積層，輸入通道數為 1，輸出通道數為 32，卷積核大小為 3x3，步幅為 1
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # 第二個卷積層，輸入通道數為 32，輸出通道數為 64，卷積核大小為 3x3，步幅為 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # 全連接層，輸入大小為 12*12*64，輸出大小為 64
        self.fc1 = nn.Linear(12 * 12 * 64, 64)
        # 輸出層，輸出大小為 10，對應 10 個類別
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        # 前向傳播
        x = torch.relu(self.conv1(x))            # 第一個卷積層 + ReLU 激活
        x = torch.max_pool2d(x, 2)               # 最大池化，池化窗口大小為 2x2
        x = torch.relu(self.conv2(x))            # 第二個卷積層 + ReLU 激活
        x = torch.max_pool2d(x, 2)               # 最大池化，池化窗口大小為 2x2
        x = x.view(-1, 12 * 12 * 64)             # 展開為一維向量
        x = torch.relu(self.fc1(x))              # 全連接層 + ReLU 激活
        x = self.fc2(x)                          # 輸出層
        return torch.softmax(x, dim=1)           # 使用 softmax 進行分類

# 初始化模型、損失函數與優化器
model = CNN()
# 使用交叉熵損失函數
criterion = nn.CrossEntropyLoss()
# 使用 Adam 優化器，學習率為 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()  # 設置模型為訓練模式
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()           # 梯度歸零
            outputs = model(images)         # 前向傳播
            loss = criterion(outputs, labels)  # 計算損失
            loss.backward()                 # 反向傳播
            optimizer.step()                # 更新權重
            running_loss += loss.item()     # 累加損失
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')  # 輸出每個 epoch 的平均損失

# 測試模型
def test(model, test_loader):
    model.eval()  # 設置模型為評估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 禁用梯度計算
        for images, labels in test_loader:
            outputs = model(images)         # 前向傳播
            _, predicted = torch.max(outputs.data, 1)  # 獲取預測結果
            total += labels.size(0)         # 累加總樣本數
            correct += (predicted == labels).sum().item()  # 累加正確預測數
    print(f'Test Accuracy: {100 * correct / total}%')  # 輸出測試準確率

# 執行訓練與測試
train(model, train_loader, criterion, optimizer, epochs=5)
test(model, test_loader)


### **TensorFlow 實作 CNN（CIFAR-10 資料集）**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10

# 下載並預處理 CIFAR-10 資料集
# CIFAR-10 是一個包含 10 種類別的彩色圖像資料集，每個類別有 6000 張圖像
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 將圖像數據轉換為浮點數並正規化到 0-1 範圍
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 建立 CNN 模型
model = models.Sequential([
    # 第一個卷積層，使用 32 個 3x3 的卷積核，激活函數為 ReLU，輸入形狀為 (32, 32, 3)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    # 最大池化層，池化窗口大小為 2x2
    layers.MaxPooling2D((2, 2)),
    # 第二個卷積層，使用 64 個 3x3 的卷積核，激活函數為 ReLU
    layers.Conv2D(64, (3, 3), activation='relu'),
    # 最大池化層，池化窗口大小為 2x2
    layers.MaxPooling2D((2, 2)),
    # 第三個卷積層，使用 64 個 3x3 的卷積核，激活函數為 ReLU
    layers.Conv2D(64, (3, 3), activation='relu'),
    # 展開層，將多維數據展開為一維
    layers.Flatten(),
    # 全連接層，包含 64 個神經元，激活函數為 ReLU
    layers.Dense(64, activation='relu'),
    # 輸出層，包含 10 個神經元，對應 10 個類別，激活函數為 softmax
    layers.Dense(10, activation='softmax')
])

# 編譯模型
# 使用 Adam 優化器，損失函數為 sparse_categorical_crossentropy，評估指標為準確率
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
# 使用訓練數據進行訓練，訓練 10 個 epoch，每批次大小為 64
model.fit(x_train, y_train, epochs=10, batch_size=64)

# 評估模型
# 使用測試數據進行評估，並輸出測試準確率
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')


### **PyTorch 實作 CNN（CIFAR-10 資料集）**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 資料轉換與預處理
# 將圖像轉換為張量並標準化到 [-1, 1] 範圍
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 下載並加載 CIFAR-10 資料集
# CIFAR-10 是一個包含 10 種類別的彩色圖像資料集，每個類別有 6000 張圖像
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 使用 DataLoader 將資料集分批次加載
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 創建 CNN 模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一個卷積層，輸入通道數為 3，輸出通道數為 32，卷積核大小為 3x3，步幅為 1
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        # 第二個卷積層，輸出通道數為 64，卷積核大小為 3x3，步幅為 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # 第三個卷積層，輸出通道數為 64，卷積核大小為 3x3，步幅為 1
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        # 全連接層，輸入大小為 4*4*64，輸出大小為 64
        self.fc1 = nn.Linear(4 * 4 * 64, 64)
        # 輸出層，輸出大小為 10，對應 10 個類別
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        # 前向傳播
        x = torch.relu(self.conv1(x))            # 第一個卷積層 + ReLU 激活
        x = torch.max_pool2d(x, 2)               # 最大池化，池化窗口大小為 2x2
        x = torch.relu(self.conv2(x))            # 第二個卷積層 + ReLU 激活
        x = torch.max_pool2d(x, 2)               # 最大池化，池化窗口大小為 2x2
        x = torch.relu(self.conv3(x))            # 第三個卷積層 + ReLU 激活
        x = torch.max_pool2d(x, 2)               # 最大池化，池化窗口大小為 2x2
        x = x.view(-1, 4 * 4 * 64)               # 展開為一維向量
        x = torch.relu(self.fc1(x))              # 全連接層 + ReLU 激活
        x = self.fc2(x)                          # 輸出層
        return torch.softmax(x, dim=1)           # 使用 softmax 進行分類

# 初始化模型、損失函數與優化器
model = CNN()
# 使用交叉熵損失函數
criterion = nn.CrossEntropyLoss()
# 使用 Adam 優化器，學習率為 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()  # 設置模型為訓練模式
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()           # 梯度歸零
            outputs = model(images)         # 前向傳播
            loss = criterion(outputs, labels)  # 計算損失
            loss.backward()                 # 反向傳播
            optimizer.step()                # 更新權重
            running_loss += loss.item()     # 累加損失
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')  # 輸出每個 epoch 的平均損失

# 測試模型
def test(model, test_loader):
    model.eval()  # 設置模型為評估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 禁用梯度計算
        for images, labels in test_loader:
            outputs = model(images)         # 前向傳播
            _, predicted = torch.max(outputs.data, 1)  # 獲取預測結果
            total += labels.size(0)         # 累加總樣本數
            correct += (predicted == labels).sum().item()  # 累加正確預測數
    print(f'Test Accuracy: {100 * correct / total}%')  # 輸出測試準確率

# 執行訓練與測試
train(model, train_loader, criterion, optimizer, epochs=10)
test(model, test_loader)


以上是基於 MNIST 和 CIFAR-10 資料集的 CNN 實作，分別使用 TensorFlow 和 PyTorch。